In [1]:
import boto3
import json

In [2]:
# get current accout and region
account = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ds_nb_exec_role = f'arn:aws:iam::{account}:role/abc.dev.all.sagemaker.data-scientist-exec.role'
ds_nb_resource = f'arn:aws:sagemaker:{region}:{account}:notebook-instance/*'
ds_policy_name = 'abc.dev.us-west-2.data-scientist.create-nb.policy'

# print(f'{ds_nb_exec_role} \n{ds_nb_resource}')

In [3]:
# Create a data scientist policy
# String condition operators: 
# https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_policies_elements_condition_operators.html#Conditions_String

ds_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": ds_nb_exec_role
        },
        {
            "Effect": "Allow",
            "Action": "sagemaker:CreateNotebookInstance",
            "Resource": ds_nb_resource,
            "Condition": {
                "ForAnyValue:StringEquals": {
                    "sagemaker:InstanceTypes": [
                        "ml.t2.medium",
                        "ml.t3.medium"
                    ]
                }
            }
        }
    ]
}

In [4]:
# Create IAM client
iam = boto3.client('iam')

response = iam.create_policy(
  PolicyName=ds_policy_name,
  PolicyDocument=json.dumps(ds_policy)
)
print(response)
policy_arn = response['Policy']['Arn']
policy_arn

{'Policy': {'PolicyName': 'abc.dev.us-west-2.data-scientist.create-nb.policy', 'PolicyId': 'ANPA26JQQ3SNCNJYLMFCH', 'Arn': 'arn:aws:iam::752257916058:policy/abc.dev.us-west-2.data-scientist.create-nb.policy', 'Path': '/', 'DefaultVersionId': 'v1', 'AttachmentCount': 0, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'CreateDate': datetime.datetime(2020, 7, 27, 12, 37, 28, tzinfo=tzlocal()), 'UpdateDate': datetime.datetime(2020, 7, 27, 12, 37, 28, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': '325cf029-6c4e-4eab-8e12-de8153b5799f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '325cf029-6c4e-4eab-8e12-de8153b5799f', 'content-type': 'text/xml', 'content-length': '833', 'date': 'Mon, 27 Jul 2020 12:37:27 GMT'}, 'RetryAttempts': 0}}


'arn:aws:iam::752257916058:policy/abc.dev.us-west-2.data-scientist.create-nb.policy'

# create data scient execution role

In [5]:
ds_role_name = 'abc.dev.us-west-2.data-scientist.basic-execution.role'

In [6]:
# https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_create_policy-examples.html
ds_assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": { "AWS": account},
            "Action": [
                "sts:AssumeRole"
            ]
        }
    ]
}

In [9]:
response = iam.create_role(
    RoleName=ds_role_name,
    AssumeRolePolicyDocument=json.dumps(ds_assume_role_policy)
)
print(response)

{'Role': {'Path': '/', 'RoleName': 'abc.dev.us-west-2.data-scientist.basic-execution.role', 'RoleId': 'AROA26JQQ3SND4744QBEI', 'Arn': 'arn:aws:iam::752257916058:role/abc.dev.us-west-2.data-scientist.basic-execution.role', 'CreateDate': datetime.datetime(2020, 7, 27, 12, 38, 38, tzinfo=tzlocal()), 'AssumeRolePolicyDocument': {'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'AWS': '752257916058'}, 'Action': ['sts:AssumeRole']}]}}, 'ResponseMetadata': {'RequestId': '0bd5d01e-db92-48cd-ad13-5335a8b0381a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0bd5d01e-db92-48cd-ad13-5335a8b0381a', 'content-type': 'text/xml', 'content-length': '854', 'date': 'Mon, 27 Jul 2020 12:38:37 GMT'}, 'RetryAttempts': 0}}


In [10]:
response = iam.attach_role_policy(RoleName=ds_role_name, PolicyArn=policy_arn)
response

{'ResponseMetadata': {'RequestId': 'f16f78f8-04df-4de0-8c8b-1d1a03045563',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f16f78f8-04df-4de0-8c8b-1d1a03045563',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 27 Jul 2020 12:38:50 GMT'},
  'RetryAttempts': 0}}